# Spatiotemporal Clustering of COVID-19 Infections in Germany

<img src="static/sars-cov-2.jpg">
(c) imago images/ZUMA Press

> **Note**: This notebook runs the [R-Kernel](https://irkernel.github.io/installation/).

## 1. About

### 1.1 COVID-19 data

This notebook visualizes and clusters COVID-19 infections published by the **Robert Koch-Institute (RKI)** in anonymized form.

The data represent the official statistics for Germany, i.e. confirmed cases that are reported from the medical authorities.

The spatial reporting unit is commonly the place of residence - not the place of the infection.

The reporting date is the date when the infection was reported to the RKI - not the date of the infection.

The data are updated daily.
To use the latest COVID-19 data, run the Jupyter Notebook `prepare_data.ipynb` before proceeding.

<img src="static/tagesschau.jpg" width=500>
(c) tagesschau

### 1.2 Clustering

We will use the classical **DBSCAN** algorithm to cluster the infections with respect to their spatial and temporal occurence.

For theory, see the presentation `material/spatio-temporal-clustering.pdf`.

<img src="static/dbscan-cluster.jpg" width=500>

For algorithm, see the paper `literature/ester_etal_kdd_1996.pdf`.

We are using the R-package `dbscan`, which provides a very fast runtime suitable for a live demo with real data. 
For details, see `literature/hahsler_etal_jss_2019.pdf`.

---

## 2. First look at the Data 

### 2.1 Read the data

We read a application-ready table that holds all entries with more than 25 reported cases to look at the hotspots of the CORONA pandemic.

In [ ]:
covid <- read.csv("data/covid19-deu_25-cases.csv")
covid$T <- as.POSIXct(covid$T)
str(covid)

> | Variable  | Description |
> |-----------|-------------|
> | X         | Longitude of district centroid |
> | Y         | Latitude  of district centroid |
> | DOY       | Day-of-Year |
> | T         | Date |
> | N         | Number of infections |
> | NAME      | District Name |

The cases are reported for 412 districts ("Landkreise" or "Stadtkreise" - in most cases). 

Optimally, we would have access to much finer data, e.g. the address.
This is not possible due to data protection policies.

Nonetheless, we are treating the spatial dimension of the reported cases as point dataset.
The table includes the district centroids as X- and Y-coordinates (decimal degree). 
Note that each case in the same district has the same coordinates.

### 2.2 Display Space and Time

The following lines generate a spatiotemporal animation with daily time steps.

First, we load a shapefile with states for having a background map for visualization:

In [ ]:
# states as background layer in maps
require(sf)     # read shapefiles

state <- read_sf("data/BL_simple.shp") # simplified shapefile for faster rendering

We use the ``plotly`` package to generate interactive maps.

In [ ]:
require(plotly) # interactive plots

In [ ]:
options(warn = -1) # disable some spurious plot_ly warning

plot_ly(height = 600) %>%

# plot the states as background layer
add_sf(data       = state, 
       type       = "scatter", 
       color      = I("grey90"), 
       stroke     = I("grey40"), 
       hoverinfo  = 'skip', 
       showlegend = FALSE) %>%

# plot the COVID-19 cases as animation
add_trace(type      = "scatter", 
          mode      = "markers", 
          x         = covid$X, 
          y         = covid$Y, 
          frame     = covid$DOY, 
          text      = paste(covid$NAME, '<br>', covid$N, 'infections<br>', covid$T), 
          hoverinfo = "text") %>% 

# some layout options
animation_opts(250) %>% #, easing = "elastic", redraw = FALSE) %>%
layout(showlegend = FALSE)

> **Note**: If the map is not fully visible, click on the blue polygon on the left margin, then on the ``...`` button.

---

## 3. Cluster Analysis

In [ ]:
require(dbscan) # dbscan clustering

### 3.1 Data scaling

<font color='blue'>**How much time equals a unit of space?**</font>

By scaling the time, we are getting the spatial and temporal dimension into the same ballpark.

14 days seems to be number that is relevant for the spread of COVID-19.

In [ ]:
diff(range(covid$X))
diff(range(covid$Y))
diff(range(covid$DOY))

In [ ]:
diff(range(covid$X))
diff(range(covid$Y))
diff(range(covid$DOY/14.0))

### 3.2 Build matrix with ST triplets

In [ ]:
mat <- cbind(covid$X, 
             covid$Y, 
             covid$DOY/14.0)

### 3.3 Find optimal parameters for DBSCAN

*a)* *k (MinPts)* can be set to the number of dimensions + 1.

*b)* *Eps* can be derived from the k-dist graph. Look for the knee!

In [ ]:
k <- 4

kNNdistplot(mat, k = k)
abline(h = seq(0, 1, 0.2), lty = 3)

### 3.4 Clustering

In [ ]:
eps <- 0.4

res <- dbscan(mat, eps = eps, minPts = k)
res

The clustering result (``res$cluster``) is a vector that matches the number of lines in the input matrix.

We can join the cluster results wih the input matrix.

In [ ]:
covid_ <- cbind(covid, 
                cluster = res$cluster)

### 3.5 Visualize clusters

In [ ]:
plot_ly(height = 650) %>%

# plot the states as background layer
add_sf(data       = state, 
       type       = "scatter", 
       color      = I("grey90"), 
       stroke     = I("grey40"), 
       hoverinfo  = 'skip', 
       showlegend = FALSE) %>%

# plot the COVID-19 clusters
add_trace(type      = "scatter", 
          mode      = "markers", 
          x         = covid_$X, 
          y         = covid_$Y, 
          split     = covid_$cluster, 
          marker    = list(size = sqrt(covid_$N), opacity = 0.1), 
          text      = paste(covid_$NAME, '<br>Cluster:', covid_$cluster), 
          hoverinfo = "text") %>% 

# hide the legend bc there are too many clusters
layout(showlegend = TRUE)

---

## 4. Display time series for clusters

### 4.1. Plot time series

In [ ]:
# read the full time series with all infections
covid_full <- read.csv("data/covid19-deu.csv")
covid_full$T <- as.POSIXct(covid_full$T)
str(covid_full)
sum(covid_full$N)

In [ ]:
# convenience function for visualizing interactive COVID-19 time series
source("./plot_covid.r")

> **Usage**
>
> plot_cases(x, cum=TRUE, incidence=TRUE, LK=NULL, timing=NULL)
> 
> | Argument  | Description |
> |-----------|-------------|
> | x         | COVID-19 data |
> | cum       | Plot cumulative cases? Needs to be TRUE or FALSE |
> | incidence | Plot incidence? Needs to be TRUE or FALSE |
> | LK        | Plot individual districts? Needs to be a vector with "Landkreis" names |
> | timing    | Superimpose a temporal window? Needs to be a vector of length 2 with minimum and maximum dates (POSIXct) |

In [ ]:
# Display the full time series for Germany 

#plot_covid(covid_full, cum = FALSE, incidence = FALSE) # absolute number of infections
#plot_covid(covid_full, incidence = FALSE) # cumulated absolute number of infections
#plot_covid(covid_full) # cumulated incidence

In [ ]:
# Display the data for some selected districts:

#selection <- c(
#    "Rhein-Hunsrück-Kreis",
#    "Trier",
#    "Trier-Saarburg")

#plot_covid(covid_full, names = selection)

### 4.2 Plot the time series of some clusters we have identified in the cluster map

In [ ]:
id <- 6:8
id <- 19
n_id <- length(id)

In [ ]:
# put the plot_ly plots in a list - bc the for loop will supress plotting otherwise
p <- vector("list", n_id)

# make a plot for each selected cluster
for (i in 1:n_id){ 
    
    # get cluster
    sub <- subset(covid_, cluster == id[i])
    
    # get all districts in this cluster
    districts <- unique(sub$NAME)

    # get the temporal limits of this cluster
    tlim  <- range(sub$T)

    # plot the time series of this cluster
    p[[i]] <- plot_covid(covid_full, names = districts, timing = tlim)

}

# display all the plots
p